In [1]:
import sys
sys.path.append("../")

from electronfactors.inputs.convert_merge_guess import convert_merge_guess
from electronfactors.model.initialise import create_cache
from electronfactors.model.iteration import iteration
from electronfactors.model.display_cutouts import display_cutouts

In [4]:
convert_merge_guess()

In [6]:
create_cache(energy=12, applicator=10)

In [ ]:
iteration(energy=12, applicator=10)

In [ ]:
display_cutouts()